# **Prelim**

## GoogleDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My\ Drive/Code/Project/

Mounted at /content/drive
/content/drive/My Drive/Code/Project


## Install Packages

In [0]:
%%capture
installed_packages = !pip list
# required_packages = ['pycuda', 'scipy', 'hiddenlayer', 'sklearn']
required_packages = ['pycuda']
for x in required_packages:
  if x not in installed_packages:
    print(x)
    null_capture = !pip install $x

print('Packages checked/installed')

# Model, Dataset and Hyperparameters

In [0]:
# MobileNetV2	|	resnet18 (34,50,101,152)	|	ConvNet	|	GoogleNet
model_name = 'resnet18'
data_name = 'CIFAR10'

model_name = model_name.lower()
WEIGHTS = 32
EPOCH_NUM = 30

NUM_WORKERS = 8
BATCH_SIZE = 16

HP_momentum = 0.9
HP_weightdecay = 0.00004

LR = 0.1 # NOTE: LR is adaptive
# Adaptive LR specific parameters
LR_PATIENCE = 3 # This is the number of epochs to observe no change in before change LR
LR_FACTOR = 0.9  # factor by which to reduce the learning rate: newLR = oldLR*LF_FACTOR

# Load Data and Model

In [4]:
import model_Load
import importlib
importlib.reload(model_Load) # forces a refresh in case of any changes to imported py package

LoadModel = model_Load.LoadModel()
# Get model-specific transforms
trainT, testT = LoadModel.transform_type(model_name)
# Load chosen dataset
train_loader, test_loader, classes = LoadModel.load_data(dataset_name=data_name,
														transform_train=trainT, 
														transform_test=testT, 
														batchsize=BATCH_SIZE, 
														numworkers=NUM_WORKERS)

import importlib
# Load chosen model
if ('mobile' in model_name and 'v2' not in model_name):
	from model_MobileNet import MobileNet as model
	print('MobileNet')
elif ('mobile' in model_name and 'v2' in model_name):
	from torchvision.models import mobilenet_v2 as model
	print('MobileNetV2')
elif ('resnet' in model_name):
	model = getattr(importlib.import_module('torchvision.models'), model_name)
	print('ResNet') 
elif 'conv' in model_name:
	from model_ConvNet import ConvNet as model
	print('ConvNet')
elif 'google' in model_name:
	from model_GoogleNet import GoogleNet as model
	print('GoogleNet')
else: 
	print('Choose a correct model type: MobileNet, MobileNetV2, GoogleNet, ConvNet')

general
Dataset: CIFAR10
Classes: 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
ResNet


# Define Loss Function

In [0]:
# criterion = nn.CrossEntropyLoss()
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
	def __init__(self, alpha = 1, gamma = 2):
		super(FocalLoss, self).__init__()
		self.alpha = alpha
		self.gamma = gamma
	
	def forward(self, inputs, target):
		BCE_loss = -F.cross_entropy(inputs, target, reduction='none') # disable reduction=mean
		pt = torch.exp(BCE_loss)
		F_loss = -self.alpha * (1 - pt) ** self.gamma * BCE_loss
		return torch.mean(F_loss)

criterion = FocalLoss(alpha=1)

# Define the *```main```* function

In [0]:
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lrs
from torchsummary import summary
from timeit import default_timer as timer
from datetime import timedelta
import math
import os
import numpy as np
from scipy import io as sio
import pycuda.driver as cuda
cuda.init()
torch.cuda.current_device() 
cuda.Device(0).name()

def main():
	# ########################################################################
	# # Define Loss function and optimizer
	# ########################################################################
	# criterion = nn.CrossEntropyLoss()
	optimizer = optim.SGD(net.parameters(), lr = LR, momentum = HP_momentum, weight_decay = HP_weightdecay)
	 # Adaptive learning rate--if accuracy (over 2 epochs) doesn't increase then reduce it by 10%
	scheduler = lrs.ReduceLROnPlateau(optimizer, 'max',
									   factor = LR_FACTOR,
									   patience = LR_PATIENCE,
									   verbose = True,
									   threshold = 1)

	# scheduler = CyclicLR(optimizer, base_lr = 0.0001, max_lr = 0.1, step_size = half_cycle)  
	########################################################################
	# Begin training
	########################################################################
	# Print model hyper-parameters and hidden parameters
	print('=' * 100)
	print('Classes: [%s]' % ', '.join(map(str, classes)))
	print('Number of Filters: ' + str(WEIGHTS))
	print('Batchsize = ' + str(BATCH_SIZE))
	print('Number of Batches = ' + str(train_iterations))
	summary(net, (3, 224, 224))
	print('=' * 100)
	test_accuracy = []
	train_accuracy = []
	train_loss = []
	# history2 = hl.History()
	# canvas2 = hl.Canvas() 
	step = (0,0)
	# net.load_state_dict(torch.load(checkpoint_path))
	for epoch in range(1, EPOCH_NUM, 1):  
		# scheduler.batch_step()  # uncomment for CyclicLR
		print('Beginning Epoch ' + str(epoch) + ' with Learning Rate = ' + str(round(get_lr(optimizer), 5)))
		running_loss, test_min_acc, total, correct = (0.0, 0.0, 0, 0)
		for i, Data in enumerate(train_loader, 0):
			step = (epoch, i)
			net.train()
			# get the inputs
			inputs, labels = Data
			# zero the parameter gradients
			optimizer.zero_grad()
			# forward + backward + optimize
			outputs = net(inputs.to(device)).to(device)
			loss = criterion(outputs, labels.to(device))
			loss.backward()
			optimizer.step()
			# print statistics
			running_loss += loss.item()
			net.eval()
			_, predicted = torch.max(outputs.data, 1)
			# Accuracy of given batch
			total += labels.size(0)
			correct += (predicted == labels.to(device)).sum().item()
			train_loss.append(running_loss / 20)
			train_accuracy.append(100.0 * correct / total)
			accuracy = 100.0 * correct / total

			if i % 200 == 0:  # print every 20 mini-batches
				print('Train: [%d, %5d] Loss: %.3f Acc: %.3f' % (epoch, i + 1, running_loss / 20, accuracy))
				running_loss = 0.0
			
		# TEST LEARNT MODEL ON TEST-SET
		correct, total = (0, 0)
		with torch.no_grad():
			net.eval()
			for Data in test_loader:
				images, labels = Data
				outputs = net(images.to(device))
				_, predicted = torch.max(outputs.data, 1)
				total += labels.to(device).size(0)
				correct += (predicted == labels.to(device)).sum().item()
		test_accuracy.append(100.0 * correct / total)
		test_ep_acc = test_accuracy[-1]
		# See if LR needs to be changed--used by ReduceLROnPlateau above
		scheduler.step(test_ep_acc)
		
		test_acc_str = '[ Epoch ' + str(epoch) + ' Test Accuracy = ' + str(round(test_ep_acc, 3)) + ' % ]'
		pad = 50 - int(round(float(len(test_acc_str) / 2)))
		print('=' * pad + test_acc_str + '=' * pad)
		# SAVE BEST MODEL
		if test_min_acc < test_ep_acc:
			test_min_acc = test_ep_acc
			torch.save(net, MODEL_SAVE_PATH + '/my_best_model.pth')
	
	np.save('test_accuracy.npy', test_accuracy);
	sio.savemat('test_accuracy.mat', mdict = {'test_accuracy': test_accuracy})
	np.save('train_accuracy.npy', train_accuracy);
	sio.savemat('train_accuracy.mat', mdict = {'train_accuracy': train_accuracy})
	np.save('train_loss.npy', train_loss);
	sio.savemat('train_loss.mat', mdict = {'train_loss': train_loss})
	
	print('Finished Training')  

# Execute

In [0]:
train_iterations = int((50000/BATCH_SIZE)) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def get_lr(optimizer):
	for param_group in optimizer.param_groups:
		return param_group['lr']    

######################################################
################ CREATE MODEL SUB-DIR ################
######################################################
img_dir = './data' # Specificy path to CIFAR-10 dataset and set download yes/no flag
MODEL_SAVE_PATH = './Output/'+ model_name
cwd_path = os.getcwd()
if cwd_path == '/content': # Check current directory isn't root
    %cd /content/drive/My\ Drive/Code/Project
mkdir_var = str(cwd_path +  '/Output/' + model_name + '/Models').replace(" ", "\ ")
print(mkdir_var)
!mkdir -p $mkdir_var
print(MODEL_SAVE_PATH)
# checkpoint_path = MODEL_SAVE_PATH + '/' + model_name + '.pth'
checkpoint_path = MODEL_SAVE_PATH + '/my_best_model.pth'



######################################################
##################### EXECUTE ########################
######################################################
if __name__ == "__main__":
    # net = model().to(device)
    # net.load_state_dict(torch.load(checkpoint_path))
    if os.path.exists(checkpoint_path):
        net = torch.load(checkpoint_path)
    else:
        net = model().to(device)
    start = timer()
    main()
    end = timer()
    execution_time = timedelta(seconds=end-start)
    print(execution_time)
    # torch.save(net.state_dict(), checkpoint_path)
    file = open(MODEL_SAVE_PATH + '/ExecutionTime.txt','w')
    file.writelines(str(execution_time)) 
    file.close() 


/content/drive/My\ Drive/Code/Project/Output/resnet18/Models
./Output/resnet18
Classes: [plane, car, bird, cat, deer, dog, frog, horse, ship, truck]
Number of Filters: 32
Batchsize = 16
Number of Batches = 3125
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

Train: [2,     1] Loss: 0.069 Acc: 25.000
Train: [2,   201] Loss: 13.072 Acc: 32.960
Train: [2,   401] Loss: 12.969 Acc: 33.370
Train: [2,   601] Loss: 12.946 Acc: 33.579
Train: [2,   801] Loss: 12.693 Acc: 33.513
Train: [2,  1001] Loss: 12.694 Acc: 33.741
Train: [2,  1201] Loss: 12.587 Acc: 34.055
Train: [2,  1401] Loss: 12.417 Acc: 34.279
Train: [2,  1601] Loss: 12.192 Acc: 34.447
Train: [2,  1801] Loss: 11.973 Acc: 34.873
Train: [2,  2001] Loss: 11.712 Acc: 35.401
Train: [2,  2201] Loss: 11.612 Acc: 35.751
Train: [2,  2401] Loss: 11.505 Acc: 36.172
Train: [2,  2601] Loss: 11.110 Acc: 36.510
Train: [2,  2801] Loss: 11.283 Acc: 36.833
Train: [2,  3001] Loss: 11.682 Acc: 36.990
=================================[ Epoch 2 Test Accuracy = 49.8 % ]=================================
Beginning Epoch 3 with Learning Rate = 0.1
Train: [3,     1] Loss: 0.071 Acc: 43.750
Train: [3,   201] Loss: 11.116 Acc: 41.573
Train: [3,   401] Loss: 10.827 Acc: 41.989
Train: [3,   601] Loss: 11.017 Acc: 42.16